In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

2025-10-06 12:42:53.887988: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759754573.915000    6743 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759754573.923177    6743 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-06 12:42:53.950757: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Load CSV file (exported from InfluxDB)
df = pd.read_csv("./data/battery_data.csv")

# Convert timestamp to datetime for time-series processing
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Simulate Time-to-Failure (assuming failure happens at the last recorded timestamp)
df["timeBeforeFailure"] = (df["timestamp"].max() - df["timestamp"]).dt.total_seconds() / 3600  # Convert to hours

# Display processed data
print(df.head())

                         timestamp  batteryId  ambientTemp  batteryCurrent  \
0 2025-10-06 11:32:27.639000+00:00          1        20.61           17.25   
1 2025-10-06 11:32:30.640000+00:00          1        20.47           19.50   
2 2025-10-06 11:32:33.639000+00:00          1        20.22           14.29   
3 2025-10-06 11:32:36.639000+00:00          1        20.47           18.09   
4 2025-10-06 11:32:39.639000+00:00          1        19.98           16.50   

   batteryTemp  batteryVoltage  currentLoad  distance    kmh  stateOfCharge  \
0        25.04           47.17        100.0      6.53  14.81         0.9019   
1        25.04           47.06        100.0      6.54  16.52         0.9017   
2        25.04           47.31        100.0      6.55  12.29         0.9016   
3        25.04           47.13        100.0      6.56  15.34         0.9014   
4        25.04           47.21        100.0      6.57  14.27         0.9013   

   stateOfHealth  timeBeforeFailure  
0        99.9496  

In [5]:
# Select Features & Target Variable
features = ["batteryTemp", "batteryCurrent", "batteryVoltage", "stateOfCharge", "stateOfHealth"]
target = "timeBeforeFailure"

X = df[features]
y = df[target]

In [6]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
# Define neural network
model = keras.Sequential([
    keras.layers.Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(1) 
])

# Compile model
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

/opt/app-root/lib64/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-10-06 12:42:58.438665: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-10-06 12:42:58.438727: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: model-training-0
2025-10-06 12:42:58.438738: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: model-training-0
2025-10-06 12:42:58.438881: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 580.82.7
2025-10-06 12:42:

In [8]:
# Train model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))

Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 0.1751 - mae: 0.3158 - val_loss: 0.0160 - val_mae: 0.1012
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0114 - mae: 0.0849 - val_loss: 0.0054 - val_mae: 0.0570
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0040 - mae: 0.0483 - val_loss: 0.0015 - val_mae: 0.0284
Epoch 4/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0011 - mae: 0.0241 - val_loss: 5.3973e-04 - val_mae: 0.0177
Epoch 5/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.8402e-04 - mae: 0.0165 - val_loss: 2.7077e-04 - val_mae: 0.0127
Epoch 6/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.9758e-04 - mae: 0.0122 - val_loss: 2.0425e-04 - val_mae: 0.0114
Epoch 7/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.4138e-04 - mae: 0.0091 - val_loss: 1.2297e-04 - val_mae: 0.0086
Epoch 8/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.3302e-04 - mae: 0.0083 - val_loss: 9.0742e-05 - val_mae: 0.0074
Epoch 9/50
30/30 ━━━━━━━━━━━━━━━━━━

In [9]:
# Make Predictions
y_pred = model.predict(X_test_scaled).flatten()

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


In [10]:
# Evaluate Model
mae = mean_absolute_error(y_test, y_pred)
print(f"Model MAE: {mae:.2f} hours before failure")

Model MAE: 0.00 hours before failure


In [11]:
# Save model as keras and scaler for inference
model.save("models/battery_ttf_model.keras")
print("Model saved as TensorFlow format in 'models/battery_ttf_model_tf.keras'")

Model saved as TensorFlow format in 'models/battery_ttf_model_tf.keras'


In [12]:
# Load model
model = tf.keras.models.load_model("models/battery_ttf_model.keras")

# Export SavedModel
tf.saved_model.save(model, "models/battery_ttf_model")

print("Model converted to SavedModel format")

INFO:tensorflow:Assets written to: models/battery_ttf_model/assets


INFO:tensorflow:Assets written to: models/battery_ttf_model/assets


Model converted to SavedModel format


In [13]:
!mo --saved_model_dir models/battery_ttf_model --output_dir models/battery_ttf_model

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
2025-10-06 12:43:31.544802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759754611.570912   12196 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759754611.578971   12196 cuda_blas.cc:1418] Unable to register cuBLAS factory